In [1]:
!pip install -q accelerate==0.33.0 bitsandbytes

In [ ]:
! pip install --upgrade transformers

In [ ]:
TOKEN = "ADD TOKEN HERE"

In [2]:
import os
import torch
# from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer

In [3]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model2_name = "sarvamai/OpenHathi-7B-Hi-v0.1-Base"
dataset_name = "mlabonne/guanaco-llama2-1k" # TODO: Change later

In [ ]:
# !sudo apt-get install expect -y # use ‘expect’ to handle interaction

In [ ]:
# TOKEN = "ADD TOKEN HERE"

In [5]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    output_hidden_states=True,
    token = TOKEN
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,token = TOKEN)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model2 = AutoModelForCausalLM.from_pretrained(
    model2_name,
    device_map="auto",
    quantization_config=bnb_config,
    output_hidden_states=True,
    token = TOKEN
)

tokenizer2 = AutoTokenizer.from_pretrained(model2_name, trust_remote_code=True,token = TOKEN)
tokenizer2.pad_token = tokenizer2.eos_token
tokenizer2.padding_side = "right" # Fix weird overflow issue with fp16 training

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [12]:
prompt = "Who the first prime minister of india?"
pipe = pipeline(task="text-generation", model=model2, tokenizer=tokenizer2, max_length=200)
result = pipe(f"{prompt}", max_new_tokens=150, do_sample=True, temperature=0.7, top_p=0.9)
print(result[0]['generated_text'])

Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Who the first prime minister of india?
ône.
Who is the first prime minister of india?
---
भारत के पहले प्रधानमंत्री जवाहरलाल नेहरू थे। He was a very important person in the country's history and helped to make India a more independent and powerful nation.


In [62]:
prompt = "Did the first prime minister of india knew Kamala Kaur?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}", max_new_tokens=150, do_sample=True, temperature=1.7, top_p=0.9)
print(result[0]['generated_text'])

Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Did the first prime minister of india knew Kamala Kaur? Who did the last one know
Kamala Kaur is a famous Indian woman, we refer to her as Sonia Gandhi (she is also a prime minister's spouse!.
Jawaharlal Nehru, the first Prime Minister of India, died in 1964. Sonia Gandhi has been active in Indian politics from the 1960s.
So, it's reasonable to assume that Jawaharlal Nehru did not know Kamala Kaur/Sonia Gandhi, as she started to involve herself in politics much later.

Now, about the last one: Narendra Modi, the current Prime Minister of India. Since Sonia Gandhi is still active in politics (though she stepped down from the position of National Congress President in 2019) and


In [6]:
enc_inp = tokenizer("What is th ", return_tensors="pt")
x = model(**enc_inp)

In [7]:
enc_inp

{'input_ids': tensor([[128000,   3923,    656,    499,   1440,    922,  47042]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
print(x)
del x

# Part 1 Exploration

## Llama 3

In [ ]:
Self_Consistency = [
    [
        "how long was jawaharlal neheru serve as prime minister",
        "did jawahar lal neheru serve 16 years as prime minister",
        "how many years are between 15 august 1947 to 27 may 1964"
    ],
    [
        "Did jawahar lal neheru serve prison sentence?",
        "Why did jawahar lal neheru went to prison?"
    ],
    [
        "Has japanese ever attacked india?",
        "Did japan attacked india in 1942?",
        "What was the calcutta blitz 1942?"
    ]
]

Fact_Checking = [
    [
        "Did jawahar lal neheru serve prison sentence?",
        "Did the first primeminister of india ever try to settle down as a barister?",
        "Did the first prime minister of india knew Kamala Kaur?"
    ]
]

In [69]:
# prompt = "Did the first prime minister of india knew Kamala Kaur?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("how long was jawaharlal neheru serve as prime minister", max_new_tokens=150, do_sample=True, temperature=1.7, top_p=0.9)
print(result[0]['generated_text'])
print("---------------------------------------------------")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("did jawahar lal neheru serve 16 years as prime minister", max_new_tokens=150, do_sample=True, temperature=0.9, top_p=0.9)
print(result[0]['generated_text'])
print("---------------------------------------------------")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("how many years are between 15 august 1947 to 27 may 1964", max_new_tokens=150, do_sample=True, temperature=0.9, top_p=0.9)
print(result[0]['generated_text'])


how long was jawaharlal neheru serve as prime minister
Jawaharlal Nehru. Jawaharlal Nehru served as the Prime Minister of India for over a period of about 18 months, from August 15, 1947, to May 27, 1948. At its second general meeting held on 22–30 August 1936, the British Socialist Sunday Herald's correspondent reported on the meeting of the All-India Socialist Conference in Kanpur. Nehru also served as the Union Minister of Home Affairs (1947–1949) and the Minister of Defence (1957–1961). Jawaharlal Nehru's term as the Prime Minister of India ended on May 27, 1948. Get premium, high … Nehru was released
---------------------------------------------------
did jawahar lal neheru serve 16 years as prime minister
no, jawaharlal nehru served as prime minister of india for 17 years, from 1947 to 1964, not 16 years.
jawaharlal nehru was the first prime minister of independent india and held the office from aug 15, 1947, until his death on may 27, 1964. he served as prime minister for 17 yea

In [70]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("Did jawahar lal neheru serve prison sentence?", max_new_tokens=150, do_sample=True, temperature=0.9, top_p=0.9)
print(result[0]['generated_text'])
print("---------------------------------------------------")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("Why did jawahar lal neheru went to prison?", max_new_tokens=150, do_sample=True, temperature=0.9, top_p=0.9)
print(result[0]['generated_text'])

Did jawahar lal neheru serve prison sentence? No.
Did jawaharlal nehru serve prison sentence? Yes.
Jawaharlal Nehru was imprisoned by the British several times during the Indian independence movement. He was first arrested in 1922 and imprisoned for nine months. He was arrested again in 1924 and served a year's imprisonment. He was arrested a third time in 1932 and imprisoned for two years. Nehru's prison sentences were a result of his involvement in the Indian independence movement and his leadership of the Indian National Congress. During his imprisonment, Nehru continued to work towards the cause of Indian independence, and his experiences in prison only strengthened his resolve to fight for the country's freedom. Nehru was eventually released from prison in 1935, and
---------------------------------------------------
Why did jawahar lal neheru went to prison??
Jawaharlal Nehru, the first Prime Minister of India, was imprisoned several times during the British Raj. He was arrested 

In [71]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("Has japanese ever attacked india?", max_new_tokens=150, do_sample=True, temperature=0.9, top_p=0.9)
print(result[0]['generated_text'])
print("---------------------------------------------------")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("Did japan attacked india in 1942?", max_new_tokens=150, do_sample=True, temperature=0.9, top_p=0.9)
print(result[0]['generated_text'])
print("---------------------------------------------------")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("What was the calcutta blitz 1942?", max_new_tokens=150, do_sample=True, temperature=0.9, top_p=0.9)
print(result[0]['generated_text'])

Has japanese ever attacked india??
There are no recorded instances of Japan attacking India directly. However, Japan did have a few instances of naval and air operations in the Indian Ocean during World War II:
1. The Indian Ocean raid: In April 1942, the Japanese launched a surprise attack on the British Eastern Fleet in the Indian Ocean. Japanese aircraft and ships attacked British and Allied ships in the Ceylon (now Sri Lanka) and Indian Ocean, sinking several ships, including the British battleship HMS Cornwall.
2. The Burma campaign: During World War II, Japan launched a series of military campaigns against British colonial India in Burma (now Myanmar) and the Indian subcontinent. Japanese forces occupied parts of Burma and parts of the North-East Frontier Agency (now the state
---------------------------------------------------
Did japan attacked india in 1942??
No, Japan did not attack India in 1942. During World War II, Japan's military expansion was primarily focused on Asia, 

In [75]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("Did jawahar lal neheru serve prison sentence?", max_new_tokens=100, do_sample=True, temperature=0.9, top_p=0.9)
print(result[0]['generated_text'])
print("---------------------------------------")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("Did the first primeminister of india ever try to settle down as a barister?", max_new_tokens=100, do_sample=True, temperature=0.9, top_p=0.9)
print(result[0]['generated_text'])
print("---------------------------------------")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe("Did the first prime minister of india knew Kamala Kaur?", max_new_tokens=100, do_sample=True, temperature=0.9, top_p=0.9)
print(result[0]['generated_text'])

Did jawahar lal neheru serve prison sentence? No, he did not serve a prison sentence. Jawaharlal Nehru was an Indian independence activist and the first Prime Minister of India. He was born on November 14, 1889, and passed away on May 27, 1964. Nehru was arrested several times by the British authorities during the Indian independence movement, but he never served a prison sentence.
Nehru was arrested in 1922, 1924, 1932, and 1942, but he was released each time due to various reasons, including his poor health and the international pressure. Nehru was also imprisoned in Ahmednagar Fort from 1944 to 1946, but he was released in 1946 after India gained
---------------------------------------
Did the first primeminister of india ever try to settle down as a barister? Answer: No. The first Prime Minister of India, Jawaharlal Nehru, was a lawyer by profession. However, he never tried to settle down as a barister. Nehru was a key figure in the Indian independence movement and became the first

In [29]:
pip install langchain faiss-cpu sentence-transformers transformers


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 51.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires ptxcompiler, which is

# # # Reference for RAG = (https://medium.com/@jiangan0808/retrieval-augmented-generation-rag-with-open-source-hugging-face-llms-using-langchain-bd618371be9d)

In [7]:
from langchain.text_splitter import CharacterTextSplitter

with open('/kaggle/input/llm-mas/RAG.tct', 'r', encoding='utf-8') as file:
    text = file.read()

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000, 
    chunk_overlap=200
)
chunks = text_splitter.split_text(text)

In [ ]:
pip install sentence_transformers

In [ ]:
pip install langchain-community

In [32]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                     model_kwargs={"device":"cpu"},
                                     encode_kwargs={"normalize_embeddings": True})

vector_store = FAISS.from_texts(chunks, hf_embeddings)

/tmp/ipykernel_36/3770698020.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NameError: name 'chunks' is not defined

In [26]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from transformers import pipeline

hf_llm = HuggingFaceHub(
    repo_id=model_name,
    huggingfacehub_api_token=TOKEN,
    model_kwargs={"temperature":0.9, "max_length":150})

rag_chain = RetrievalQA.from_chain_type(
    llm=hf_llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [ ]:
query = "Your question here"
answer = rag_chain.run(query)
print(answer)

In [21]:
result = rag_chain.run("how long was jawaharlal neheru serve as prime minister")
print(result)
print("---------------------------------------------------")
result =  rag_chain.run("did jawahar lal neheru serve 16 years as prime minister")
print(result)
print("---------------------------------------------------")
result = rag_chain.run("how many years are between 15 august 1947 to 27 may 1964")
print(result)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Morarji Desai was a nationalist with anti-corruption leanings but was socially conservative, pro-business, and in favour of free enterprise reforms, as opposed to Prime Minister Jawaharlal Nehru's socialistic policies. After serving as chief minister of Bombay State, he joined Nehru's cabinet in 1956 as the finance minister of India. he held that position until 1963 when he along with other senior ministers in the Nehru cabinet resigned under the Kamaraj plan.The plan, as proposed by Madras Chief Minister K.Kamaraj, was to revert government ministers to party positions after a certain tenure and vice versa. With Nehru's age and health failing in the early 1960s, Desai was considered a possible contender for the position of Prime Minister.[272][273] Later Desai alleged that Nehru used the Kamaraj Plan to remove all possible c

In [22]:
result = rag_chain.run("Did jawahar lal neheru serve prison sentence?")
print(result)
print("---------------------------------------------------")
result =  rag_chain.run("Why did jawahar lal neheru went to prison?")
print(result)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In prison 1930-1935
On 11 October 1930, Nehru's detention ended, but he was back in jail in less than ten days for resuming the presidency of the banned Congress.[66] On 26 January 1931, Nehru and other prisoners were released early by Lord Irwin, who was negotiating with Gandhi.[67] His father died on 6 February 1931. Nehru was back in jail on 26 December 1931 after violating court orders not to leave Allahabad while leading a "no-rent" campaign to alleviate peasant distress.[68] On 30 August 1933, Nehru was released from prison, but the government soon moved to detain him again. On 22 December 1933, the Home Secretary sent a memo to all local governments in India:
    The Government of India regard him [Nehru] as by far the most dangerous element at large in India, and their view is that the time has come, in accordance wi

In [23]:
result = rag_chain.run("Did jawahar lal neheru serve prison sentence?")
print(result)
print("---------------------------------------------------")
result =  rag_chain.run("Did the first primeminister of india ever try to settle down as a barister?")
print(result)
print("---------------------------------------------------")
result = rag_chain.run("Did the first prime minister of india knew Kamala Kaur?")
print(result)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In prison 1930-1935
On 11 October 1930, Nehru's detention ended, but he was back in jail in less than ten days for resuming the presidency of the banned Congress.[66] On 26 January 1931, Nehru and other prisoners were released early by Lord Irwin, who was negotiating with Gandhi.[67] His father died on 6 February 1931. Nehru was back in jail on 26 December 1931 after violating court orders not to leave Allahabad while leading a "no-rent" campaign to alleviate peasant distress.[68] On 30 August 1933, Nehru was released from prison, but the government soon moved to detain him again. On 22 December 1933, the Home Secretary sent a memo to all local governments in India:
    The Government of India regard him [Nehru] as by far the most dangerous element at large in India, and their view is that the time has come, in accordance wi

# # OpenHathi

In [ ]:
Self_Consistency = [
    [
        "did jawahar lal nehru know go to law school?",
    ],
    [
        "how were indra gandhi and jawahar lal neheru related?",
    ],
    [
        "which law school did jawahar lal nehru go to?",
    ]
]

Fact_Checking = [
    [
        "which law school did jawahar lal nehru go to?",
        "was jawahar lal neheru part of indian national congress?",
        "was jawahar lal neheru part part of muslim league?"
    ]
]

In [20]:
pipe = pipeline(task="text-generation", model=model2, tokenizer=tokenizer2)
result = pipe("did jawahar lal nehru know go to law school?", max_new_tokens=150, temperature=0.6, top_p=0.9)
print(result[0]['generated_text'])
print("---------------------------------")
pipe = pipeline(task="text-generation", model=model2, tokenizer=tokenizer2)
result = pipe("did jawahar lal nehru know go to law school?", max_new_tokens=150, temperature=0.6, top_p=0.9)
print(result[0]['generated_text'])

did jawahar lal nehru know go to law school?
 hopefully, he will answer this question.

vikalpah
 --yes
 --nahin
---
To determine if Jawahar Lal Nehru went to law school, we need to look for information about his education and career.

charan 1: jaahir hai lal nehru ke baare mein jaankaari praapt karein.
Step 2: Search for information about his education and career.
charan 3: unki shiksha aur kariyar ke baare mein jaankaari ke aadhaar par nishkarsh nikaalein.

Step 1: Obtaining information about Jaw
---------------------------------
did jawahar lal nehru know go to law school?
 hopefully you can find out.


In [21]:
pipe = pipeline(task="text-generation", model=model2, tokenizer=tokenizer2)
result = pipe("how were indra gandhi and jawahar lal neheru related?", max_new_tokens=150, temperature=0.6, top_p=0.9)
print(result[0]['generated_text'])
print("---------------------------------")
pipe = pipeline(task="text-generation", model=model2, tokenizer=tokenizer2)
result = pipe("how were indra gandhi and jawahar lal neheru related?", max_new_tokens=150, temperature=0.6, top_p=0.9)
print(result[0]['generated_text'])

how were indra gandhi and jawahar lal neheru related?
 Einzelnा में एक आदमी की मृत्यु हो गई।
 1. १९९६ में भारत सरकार ने "विरासत अधिनियम" के तहत एक नए नियम की घोषणा की।
 2. 2017 में भारत सरकार ने "विरासत अधिनियम" के तहत एक नए नियम की घोषणा की।
 3. 2000 में भारत सरकार ने "विरासत अधिनियम" के तहत एक नए नियम की घोषणा की।
 4. 2007 में भारत सरकार ने "विरासत अधिनियम" के तहत एक नए नियम की घोषणा की।
 5. 2016 में भारत सरकार ने "वि
---------------------------------
how were indra gandhi and jawahar lal neheru related?
 everybody knows that the prime minister of india is the head of the government of india.

is savaal ka javaab haih
---
Yes, Indira Gandhi and Jawahar Lal Nehru were related. ve donon bhaarat ke pradhaanamantri the, jiska arth hai ki ve donon bhaarat ke sarkaar ke pramukh the.


In [23]:
pipe = pipeline(task="text-generation", model=model2, tokenizer=tokenizer2)
result = pipe("which law school did jawahar lal nehru go to?", max_new_tokens=150, temperature=0.6, top_p=0.9)
print(result[0]['generated_text'])

print("----------------------------------")
pipe = pipeline(task="text-generation", model=model2, tokenizer=tokenizer2)
result = pipe("which law school did jawahar lal nehru go to?", max_new_tokens=150, temperature=0.6, top_p=0.9)
print(result[0]['generated_text'])

which law school did jawahar lal nehru go to?
 everyone: huh?

जवाबः
---
Jawahar Lal Nehru went to the University of Cambridge.
----------------------------------
which law school did jawahar lal nehru go to?
 nobody knows...


In [25]:
pipe = pipeline(task="text-generation", model=model2, tokenizer=tokenizer2)
result = pipe("was jawahar lal neheru part of indian national congress?", max_new_tokens=150, temperature=0.6, top_p=0.9)
print(result[0]['generated_text'])

was jawahar lal neheru part of indian national congress?
 nobody knows
---
jab jahaan lal neheru ka janma hua tha, tab bhaarteey raashtreey kaangres ka gathan nahin hua tha. The Indian National Congress was formed later, in 1885. isliye, jahaan lal neheru ka janma hua tha, us samay bhaarteey raashtreey kaangres nahin thi.


In [27]:
pipe = pipeline(task="text-generation", model=model2, tokenizer=tokenizer2)
result = pipe("was jawahar lal neheru part part of muslim league?", max_new_tokens=150, temperature=0.6, top_p=0.9)
print(result[0]['generated_text'])

was jawahar lal neheru part part of muslim league?
 संतुलित.
---
haan, javaahar lal neheru paart muslim leeg ka ek pramukh sadasya tha. He was a member of the All-India Muslim League, which was a political party in India during the pre-independence period. muslim leeg ka uddeshya bhaarat mein musalmaano ke hiton ki raksha karna aur unhein samaan adhikaaron ki maang karna tha. 

Jawahar Lal Nehru was a prominent leader of the Indian National Congress, a rival party to the Muslim League. ve ek bhaarteey raajn


In [33]:
from langchain.text_splitter import CharacterTextSplitter

with open('/kaggle/input/llm-mas/RAG.tct', 'r', encoding='utf-8') as file:
    text = file.read()

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000, 
    chunk_overlap=200
)
chunks = text_splitter.split_text(text)

In [35]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                     model_kwargs={"device":"cpu"},
                                     encode_kwargs={"normalize_embeddings": True})

vector_store = FAISS.from_texts(chunks, hf_embeddings)

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from transformers import pipeline

hf_llm = HuggingFaceHub(
    repo_id=model2_name,
    huggingfacehub_api_token=TOKEN,
    model_kwargs={"temperature":0.9, "max_length":150})

rag_chain = RetrievalQA.from_chain_type(
    llm=hf_llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

/tmp/ipykernel_36/3018925821.py:14: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  hf_llm = HuggingFaceHub(


In [36]:
result = rag_chain.run("did jawahar lal nehru know go to law school?")
print(result)
print("---------------------------------------------------")
result =  rag_chain.run("did jawahar lal nehru know go to law school?")
print(result)

/tmp/ipykernel_36/1119755770.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = rag_chain.run("did jawahar lal nehru know go to law school?")


HfHubHTTPError:  (Request ID: 52cUk4Zgwxpa-zY2iVrai)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/sarvamai/OpenHathi-7B-Hi-v0.1-Base.
If you are trying to create or update content, make sure you have a token with the `write` role.
The model sarvamai/OpenHathi-7B-Hi-v0.1-Base is too large to be loaded automatically (13GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).

In [ ]:
result = rag_chain.run("how long was jawaharlal neheru serve as prime minister")
print(result)
print("---------------------------------------------------")
result =  rag_chain.run("did jawahar lal neheru serve 16 years as prime minister")
print(result)

In [ ]:
result = rag_chain.run("how long was jawaharlal neheru serve as prime minister")
print(result)
print("---------------------------------------------------")
result =  rag_chain.run("did jawahar lal neheru serve 16 years as prime minister")
print(result)

In [ ]:
result = rag_chain.run("how long was jawaharlal neheru serve as prime minister")
print(result)
print("---------------------------------------------------")
result =  rag_chain.run("did jawahar lal neheru serve 16 years as prime minister")
print(result)
print("---------------------------------------------------")
result =  rag_chain.run("did jawahar lal neheru serve 16 years as prime minister")
print(result)

# Part 2 Probing

In [ ]:
! pip install datasets

In [8]:
from datasets import load_dataset

# Load a dataset by its name
dataset = load_dataset("michaelauli/wiki_bio",split="train[:20000]")

# train_data = dataset['train']
# test_data = dataset['test']

The repository for michaelauli/wiki_bio contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/michaelauli/wiki_bio.

You can avoid this prompt in future by passing the argument `trust_remote_code=True`.



Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/582659 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/72831 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/72831 [00:00<?, ? examples/s]

In [9]:
train_data = dataset

In [10]:
import re
import numpy as np
x = dict()
t = dict()
context = []
nat = []
birth_year = []
for i in range(20000):
    # context.append(re.sub("-rrb-",")",re.sub("-lrb-","(", train_data[i]['input_text']['context'])))
    for j in train_data[i]['input_text']['table']['column_header']:
        try:
            x[j] += 1
        except:
            x[j] = 1
    try:
        s = list(train_data[i]['input_text']['table']['column_header']).index('nationality')
        f = list(train_data[i]['input_text']['table']['column_header']).index('birth_date')
        try:
            t[train_data[i]['input_text']['table']['content'][s].split()[0][:2]] += 1
        except:
            t[train_data[i]['input_text']['table']['content'][s].split()[0][:2]] = 1
        # t.add(train_data[i]['input_text']['table']['content'][s].split()[0][:2])
    except ValueError:
        continue

key = np.array(list(t.keys()),dtype=str)
val = np.array(list(t.values()))
vls = np.argsort(val)

In [11]:
val[vls[::-1][:10]]

array([818, 326, 166, 131, 109, 107, 106, 105,  93,  83])

In [12]:
classes  = set([str(i) for i in key[vls[::-1][:10]]])
print(classes)

{'un', 'in', 'it', 'ca', 'br', 'en', 'fr', 'au', 'am', 'ge'}


In [102]:
train_context = []
train_by = []
train_nat = []

test_context = []
test_by = []
test_nat = []
natn = dict()
for i in range(20000):
    try:
        s = list(train_data[i]['input_text']['table']['column_header']).index('nationality')
        nt = train_data[i]['input_text']['table']['content'][s].split()[0][:2]
        if(nt not in classes):
            continue
        f = list(train_data[i]['input_text']['table']['column_header']).index('birth_date')
        bt = train_data[i]['input_text']['table']['content'][f]
        bt2 = re.search(r"\d{4}",bt)
        if bt2  == None:
            continue
        bt3 = bt[bt2.start():bt2.end()]
        cnt = re.sub("-rrb-",")",re.sub("-lrb-","(", train_data[i]['input_text']['context']))
        try:
            if(natn[nt] == 50):
                continue
            if(natn[nt] >= 30):
                test_nat.append(nt)
                test_by.append(int(bt3))
                test_context.append(cnt.strip())
            else:
              train_nat.append(nt)
              train_by.append(int(bt3))
              train_context.append(cnt.strip())
            natn[nt] += 1
        except KeyError:
            train_nat.append(nt)
            train_by.append(int(bt3))
            train_context.append(cnt.strip())
            natn[nt] = 1

    except ValueError:
        continue

In [103]:
import pandas as pd

train_df = pd.DataFrame({"context":train_context, "birth_year":train_by, "nationality":train_nat})
train_df.to_csv("train.csv")
test_df = pd.DataFrame({"context":test_context, "birth_year":test_by, "nationality":test_nat})
test_df.to_csv("test.csv")

In [104]:
train_df

,context,birth_year,nationality
0,walter extra,1954,ge
1,italo mus,1892,it
2,herbie kronowitz,1923,am
3,arthur arnould,1833,fr
4,ward williams,1923,am
...,...,...,...
295,william curtis green,1875,en
296,"sir giles rolls loder , 3rd baronet",1914,en
297,gerhart hauptmann,1862,ge
298,georg hackl,1966,ge


In [105]:
test_df

,context,birth_year,nationality
0,raymond c. hoiles,1878,am
1,david west ( basketball ),1980,am
2,damen bell-holter,1990,am
3,john steptoe,1950,am
4,margaret kilgallen,1967,am
...,...,...,...
195,warren dunford,1963,ca
196,robert mustard ( martial artist ),1956,ca
197,j. r. williams,1888,ca
198,anthony hyde,1946,ca


In [106]:
layer = [1,16,32]

In [107]:
train_first = []
train_last = []
train_middle = []
train_by = []
train_nationality = []
train_context = []


In [108]:
from tqdm import tqdm

for index, row in tqdm(train_df.iterrows()):
  enc_inp = tokenizer("Give details of year of birth and nationality of " + row["context"],return_tensors="pt")
  # enc_inp = tokenizer("what is the nationality of " + row["context"],return_tensors="pt")
  x = model(**enc_inp)
  train_first.append(x["hidden_states"][layer[0]][0][-1].detach().cpu().numpy())
  train_middle.append(x["hidden_states"][layer[1]][0][-1].detach().cpu().numpy())
  train_last.append(x["hidden_states"][layer[2]][0][-1].detach().cpu().numpy())
  del x
  train_by.append(int(row["birth_year"]))
  train_nationality.append(row["nationality"])
  train_context.append(row["context"])
  # break

300it [03:35,  1.39it/s]


In [109]:
test_first = []
test_last = []
test_middle = []
test_by = []
test_nationality = []
test_context = []

for index, row in tqdm(test_df.iterrows()):
  enc_inp = tokenizer("Give details of year of birth and nationality of " + row["context"],return_tensors="pt")
  # enc_inp = tokenizer("what is the nationality of " + row["context"],return_tensors="pt")
  x = model(**enc_inp)
  test_first.append(x["hidden_states"][layer[0]][0][-1].detach().cpu().numpy())
  test_middle.append(x["hidden_states"][layer[1]][0][-1].detach().cpu().numpy())
  test_last.append(x["hidden_states"][layer[2]][0][-1].detach().cpu().numpy())
  del x
  test_by.append(int(row["birth_year"]))
  test_nationality.append(row["nationality"])
  test_context.append(row["context"])
  # break

200it [02:23,  1.39it/s]


In [110]:
import torch
torch.cuda.empty_cache()


In [111]:
test_dff = pd.DataFrame({
    "a":test_context,
    "b":test_nationality,
    "c":test_by,
    "d":test_first,
    "e":test_middle,
    "f":test_last
    })

train_dff = pd.DataFrame({
    "a":train_context,
    "b":train_nationality,
    "c":train_by,
    "d":train_first,
    "e":train_middle,
    "f":train_last
    })

In [112]:
import pandas as pd
import pickle
# pickle.dump(train_dff,)
with open(r"test.pickle", "wb") as output_file:
  pickle.dump(test_dff, output_file)

with open(r"test.pickle", "rb") as in_file:
  x = pickle.load(in_file)

with open(r"train.pickle", "wb") as output_file:
  pickle.dump(train_dff, output_file)

with open(r"train.pickle", "rb") as in_file:
  x = pickle.load(in_file)
# test_dff = pd.read_json("test.json")
# train_dff = pd.read_json("train.json")

In [24]:

# x["d"][0][0]

,a,b,c,d,e,f
0,walter extra,ge,1954,"[0.001656, -0.001045, -0.01459, -0.004395, -0....","[0.1265, -0.0601, 0.05753, 0.0929, -0.01453, 0...","[4.0, -2.521, 1.04, 2.684, -1.286, 1.9375, -5...."
1,italo mus,it,1892,"[-0.02063, -0.00775, -0.01624, 0.008606, 0.001...","[0.05725, 0.0653, -0.05353, 0.0706, 0.03693, 0...","[-0.64, 0.6895, 2.95, -0.4907, -1.942, 2.936, ..."
2,herbie kronowitz,am,1923,"[-0.00209, -0.0076, 0.01758, 0.00946, -0.00723...","[-0.00464, 0.1487, 0.1294, 0.058, -0.06183, -0...","[-2.861, -0.951, 2.064, 1.585, 2.354, 3.48, -6..."
3,arthur arnould,fr,1833,"[0.01166, -0.01105, 0.004425, -0.007812, -0.01...","[-0.1186, 0.0636, -0.06158, 0.1711, 0.01996, -...","[-1.81, 1.377, 0.6987, 1.862, 2.53, 1.321, -4...."
4,ward williams,am,1923,"[-0.01355, -0.00842, 0.003464, 0.01276, 0.0064...","[-0.08136, 0.002258, -0.0891, 0.0101, 0.117, -...","[-2.441, -1.4795, 1.619, 1.973, 3.254, 4.535, ..."
...,...,...,...,...,...,...
495,warren dunford,ca,1963,"[-0.00656, 0.0005455, 0.003891, 0.0008698, -0....","[-0.01959, 0.04016, -0.02716, 0.07556, 0.2166,...","[-1.456, -1.734, 1.276, 2.285, 4.633, 4.098, -..."
496,robert mustard ( martial artist ),ca,1956,"[-0.01636, 0.007324, -0.001968, 0.0011215, -0....","[0.08405, 0.2036, -0.0386, 0.04968, 0.003906, ...","[-3.299, -2.307, 0.7676, 3.336, 2.855, -1.317,..."
497,j. r. williams,ca,1888,"[-0.01355, -0.00842, 0.003464, 0.01276, 0.0064...","[-0.09973, 0.0791, -0.0945, 0.0823, 0.2444, 0....","[-2.422, -2.531, 1.425, 2.98, 2.797, 3.637, -5..."
498,anthony hyde,ca,1946,"[-0.00934, -0.001671, 0.003143, -0.005157, -0....","[-0.1266, -0.001709, -0.01846, 0.02417, 0.1941...","[-1.096, 1.593, 0.1333, 1.363, 0.3142, 5.215, ..."


In [113]:
tX_0 = np.array([list(i) for  i in train_dff["d"].to_list()])
tX_1 = np.array([list(i) for  i in  train_dff["e"].to_list()])
tX_2 = np.array([list(i) for  i in  train_dff["f"].to_list()])
ty_c = train_dff["b"].to_numpy()
ty_r = train_dff["c"].to_numpy()

!pip install -qU langchain langchain-openai langchain_chroma langchain_community langchainhub
!pip install -qU langchain langchain-openai langchain_chroma langchain_community langchainhub
!pip install -qU langchain langchain-openai langchain_chroma langchain_community langchainhub


In [75]:
# tX_2

In [114]:
from sklearn import svm
clf0 = svm.SVC(decision_function_shape='ovo')
clf0.fit(tX_0, ty_c)
clf1 = svm.SVC(decision_function_shape='ovo')
clf1.fit(tX_1, ty_c)
clf2 = svm.SVC(decision_function_shape='ovo')
clf2.fit(tX_2, ty_c )

SVC(decision_function_shape='ovo')

In [115]:
typ0_c = clf0.predict(tX_0)
typ1_c = clf1.predict(tX_1)
typ2_c = clf2.predict(tX_2)

In [116]:
from sklearn.metrics import accuracy_score, confusion_matrix
print("Train")
print("Layer 0 acuracy: ", accuracy_score(ty_c,typ0_c))
print("Layer 1 acuracy: ", accuracy_score(ty_c,typ1_c))
print("Layer 2 acuracy: ", accuracy_score(ty_c,typ2_c))

Train

Layer 0 acuracy:  0.8866666666666667

Layer 1 acuracy:  0.8266666666666667

Layer 2 acuracy:  0.75


In [117]:
teX_0 = np.array([list(i) for  i in  test_dff["d"].to_list()])
teX_1 = np.array([list(i) for  i in  test_dff["e"].to_list()])
teX_2 = np.array([list(i) for  i in  test_dff["f"].to_list()])
tey_c = test_dff["b"].to_numpy()
tey_r = test_dff["c"].to_numpy()

In [118]:
typ0_c = clf0.predict(teX_0)
typ1_c = clf1.predict(teX_1)
typ2_c = clf2.predict(teX_2)

print("Test")
print("Layer 0 acuracy: ", accuracy_score(tey_c,typ0_c))
print("Layer 1 acuracy: ", accuracy_score(tey_c,typ1_c))
print("Layer 2 acuracy: ", accuracy_score(tey_c,typ2_c))

Test

Layer 0 acuracy:  0.275

Layer 1 acuracy:  0.405

Layer 2 acuracy:  0.45


In [119]:
from sklearn.linear_model import LinearRegression
clf0 = LinearRegression()
clf0.fit(tX_0, ty_r)
clf1 = LinearRegression()
clf1.fit(tX_1, ty_r)
clf2 = LinearRegression()
clf2.fit(tX_2, ty_r )


LinearRegression()

In [123]:
from sklearn.metrics import r2_score

typ0_r = clf0.predict(tX_0)
typ1_r = clf1.predict(tX_1)
typ2_r = clf2.predict(tX_2)

print("Training")
print("Layer 0 acuracy: ", mean_squared_error(ty_r,typ0_r))
print("Layer 1 acuracy: ", mean_squared_error(ty_r,typ1_r))
print("Layer 2 acuracy: ", mean_squared_error(ty_r,typ2_r))

Training

Layer 0 acuracy:  0.13333333333333333

Layer 1 acuracy:  0.0

Layer 2 acuracy:  0.0


In [124]:
from sklearn.metrics import mean_squared_error
typ0_r = clf0.predict(teX_0)
typ1_r = clf1.predict(teX_1)
typ2_r = clf2.predict(teX_2)

print("Test")
print("Layer 0 acuracy: ", mean_squared_error(tey_r,typ0_r))
print("Layer 1 acuracy: ", mean_squared_error(tey_r,typ1_r))
print("Layer 2 acuracy: ", mean_squared_error(tey_r,typ2_r))

Test

Layer 0 acuracy:  26347.13

Layer 1 acuracy:  6634.47

Layer 2 acuracy:  7990.37
